In [1]:
%cd ../
%pwd

/home/timur.bikbulatov/personal/aa_on_vad


/home/timur.bikbulatov/miniconda3/envs/aaml/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/home/timur.bikbulatov/personal/aa_on_vad'

In [2]:
import soundfile as sf
from pathlib import Path
import numpy as np
import torch
import IPython
from functools import partial
play = partial(IPython.display.Audio,
               rate=16000)
playr = IPython.display.Audio

In [3]:
import plotly.graph_objects as go


def plot(y:list):
    trace = []
    colors = [
        'Blue',
        'Orange',
        'Green',
        'Red',
        'Purple',
        'Magenta',
        'Cyan',
        'Brown',
        'Pink',
        'Lime',
        'Yellow',
        'Teal',
        'Olive',
        'Navy',
        'Maroon',
        'Coral',
        'Gold',
        'Indigo',
        'Turquoise',
        'Lavender',
        'Mint',
        'Silver',
        ]
    for ik, y_ in enumerate(y):
        trace.append(go.Scatter(x=np.arange(len(y_)), y=y_, mode='lines', name=f'arg # {ik + 1}', line=dict(color=colors[ik])))

    # Combining both traces into one figure
    fig = go.Figure(data=trace)

    # Setting the layout
    fig.update_layout(
        title='Two Line Charts on One Plot',
        xaxis_title='X-axis',
        yaxis_title='Y-axis',
        showlegend=True
    )

    # Display the plot
    fig.show()

In [4]:
from src.datasets.mixed_vad_datset import get_datset
from src.models.adversarial_models.CNN1d_based import NoiseGenerator
from src.models.silero.evaluate_model import validate_silero_vad, print_metrics
import src.models.silero.train as tl
from src.models.silero.loss import AdversarialLoss

In [5]:
print('Loading dataset...')
train_ds = get_datset(mode='train', erase_silence=True)
eval_ds = get_datset(mode='eval', erase_silence=True)
print('Dataset loaded!')

Loading dataset...
Dataset loaded!


In [6]:
model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad', model='silero_vad')
print(model)

RecursiveScriptModule(
  original_name=VADRNNJITMerge
  (_model): RecursiveScriptModule(
    original_name=VADRNNJIT
    (stft): RecursiveScriptModule(
      original_name=STFT
      (padding): RecursiveScriptModule(original_name=ReflectionPad1d)
    )
    (encoder): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(
        original_name=SileroVadBlock
        (se): RecursiveScriptModule(original_name=Identity)
        (activation): RecursiveScriptModule(original_name=ReLU)
        (reparam_conv): RecursiveScriptModule(original_name=Conv1d)
      )
      (1): RecursiveScriptModule(
        original_name=SileroVadBlock
        (se): RecursiveScriptModule(original_name=Identity)
        (activation): RecursiveScriptModule(original_name=ReLU)
        (reparam_conv): RecursiveScriptModule(original_name=Conv1d)
      )
      (2): RecursiveScriptModule(
        original_name=SileroVadBlock
        (se): RecursiveScriptModule(original_name=Identity)
     

Using cache found in /home/timur.bikbulatov/.cache/torch/hub/snakers4_silero-vad_master


In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [8]:
amodel = NoiseGenerator().to(device)

In [12]:
metrics = validate_silero_vad(
        model,
        eval_ds,
        1,
        # noise_producer,
    )
print_metrics(metrics)

  0%|          | 1/262 [00:00<00:59,  4.36it/s]/home/timur.bikbulatov/personal/aa_on_vad/src/datasets/urbansound.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  audio = torch.tensor(self.audios[idx], dtype=torch.float32)
100%|██████████| 262/262 [00:13<00:00, 18.73it/s]


precision: 0.8369
recall: 0.9049
f1_score: 0.8696
accuracy: 0.8867
false_positive_rate: 0.1264
true_positive_rate: 0.9049
roc_auc: 0.8892


In [61]:
import importlib
importlib.reload(tl)

<module 'src.models.silero.train' from '/home/timur.bikbulatov/personal/aa_on_vad/src/models/silero/train.py'>

In [9]:
tl.train_noise_producer(
        amodel,
        model,
        train_ds,
        eval_ds,
        epochs=3,
        criterion=AdversarialLoss(alpha=10)
    )

Noise producer model has 25903393 parametes


Epoch 0:   0%|          | 0/66 [00:00<?, ?it/s]

/home/timur.bikbulatov/personal/aa_on_vad/src/datasets/urbansound.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  audio = torch.tensor(self.audios[idx], dtype=torch.float32)
Epoch 0: 100%|██████████| 66/66 [03:28<00:00,  3.16s/it, loss=0.0177]


Epoch 0: Train Loss = 0.0311


Evaluation: 100%|██████████| 9/9 [00:12<00:00,  1.37s/it]


...

Epoch 1: Eval Loss = 0.0038


Epoch 2: 100%|██████████| 66/66 [03:28<00:00,  3.17s/it, loss=-0.0184]


Epoch 2: Train Loss = -0.0012


Evaluation: 100%|██████████| 9/9 [00:12<00:00,  1.38s/it]

Epoch 2: Eval Loss = -0.1288


In [10]:
metrics = validate_silero_vad(
        model,
        eval_ds,
        32,
        amodel,
    )
print_metrics(metrics)

100%|██████████| 9/9 [00:30<00:00,  3.34s/it]


precision: 0.7866
recall: 0.4122
f1_score: 0.5409
accuracy: 0.7079
false_positive_rate: 0.0802
true_positive_rate: 0.4122
roc_auc: 0.6660


In [ ]:
i=9
wav = eval_ds[i]['sample']
mask = eval_ds[i]['mask']
noise = amodel(wav.unsqueeze(0).to(device)).detach().cpu()
plot([wav, mask, noise[0], wav+noise[0]])

In [ ]:
play(wav+amodel(wav.unsqueeze(0).to(device)).detach().cpu())

In [ ]:
play(noise)

In [67]:
wav.shape

torch.Size([96000])